In [4]:
import pandas as pd
import numpy as np
import ast
from langdetect import detect
from tabulate import tabulate


### 1. CARGA DE DATOS

In [5]:
# Se lee el archivo output_steam_games.json que esta definido como un unico objeto json y se define el dataframe correspondiente

df_games = pd.read_json('./Datasets/output_steam_games.json', lines=True)
print(df_games.head())   # Se observan una gran cantidad de valores nulos en la filas iniciales

#print(df_games.tail())


  publisher genres app_name title   url release_date  tags reviews_url specs  \
0      None   None     None  None  None         None  None        None  None   
1      None   None     None  None  None         None  None        None  None   
2      None   None     None  None  None         None  None        None  None   
3      None   None     None  None  None         None  None        None  None   
4      None   None     None  None  None         None  None        None  None   

  price  early_access  id developer  
0  None           NaN NaN      None  
1  None           NaN NaN      None  
2  None           NaN NaN      None  
3  None           NaN NaN      None  
4  None           NaN NaN      None  


In [6]:
# Se leen los otros 2 archivos json, donde cada línea del archivo contiene un objeto json

def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)

df_reviews = load_json_lines("./Datasets/australian_user_reviews.json")
df_items = load_json_lines("./Datasets/australian_users_items.json")

In [ ]:
# Vista preliminar de los datos
#print(df_reviews.head())
#print(df_reviews.tail())

#print(df_items.head())
#print(df_items.tail())

with pd.option_context('display.max_colwidth', None):
    print(df_reviews.head(5))  # Se observa que un mismo usuario puede tener varias recomendaciones en una misma fila    


In [ ]:
# Vista preliminar de los datos
#print(df_reviews.info())
#print(df_items.info())
print(df_games.shape)
print(df_games.info())    # Se observan una gran cantidad de datos nulos


In [9]:
print(df_games.isnull().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


In [10]:
# Se eliminan las filas del dataframe que tienen todos sus valores nulos
df_games = df_games.dropna(how='all')
print(df_games.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


### 2. ETL-REVIEWS

In [11]:
# Se expandira el dataframe df_reviews para que quede una fila por cada recomendación 
# Y una columna por cada dato de la seria almacenada en la columna de 'review'

expanded_rows = []
for index, row in df_reviews.iterrows():
    user_id  = row['user_id']
    user_url = row['user_url']

    review_list = row['reviews']

    for it in review_list:
        # se genera diccionario del nuevo df expandido
        new_item = {    'user_id'  : user_id ,
                        'user_url' : user_url,
                        **it
                    }
        expanded_rows.append(new_item)

df_reviews_exp = pd.DataFrame(expanded_rows)
#df_reviews_exp.to_csv(r'./Datasets/reviews_exp.csv',index=False)

In [12]:
with pd.option_context('display.max_colwidth', None):
    print(df_reviews_exp.head(5))      # Se observa una gran cantidad de datos nulos o vacios en columna 'last_edited' 
    #print(df_reviews_exp.tail(5))

print(df_reviews_exp.shape)


             user_id                                              user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
1  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
2  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
3            js41637                  http://steamcommunity.com/id/js41637   
4            js41637                  http://steamcommunity.com/id/js41637   

  funny                     posted last_edited item_id  \
0         Posted November 5, 2011.                1250   
1            Posted July 15, 2011.               22200   
2           Posted April 21, 2011.               43110   
3            Posted June 24, 2014.              251610   
4        Posted September 8, 2013.              227300   

                                           helpful  recommend  \
0                                   No ratings yet       True   
1                                   No ratings yet  

In [13]:
print(df_reviews_exp['last_edited'].value_counts())


last_edited
                                  53165
Last edited November 25, 2013.       99
Last edited October 17, 2015.        19
Last edited June 6, 2015.            18
Last edited January 3.               17
                                  ...  
Last edited May 30, 2015.             1
Last edited May 21, 2015.             1
Last edited February 11, 2014.        1
Last edited May 8, 2014.              1
Last edited August 15, 2014.          1
Name: count, Length: 1015, dtype: int64


In [14]:
print('Porcentaje de valores vacios de la columna last_edited: ', 53165*100/df_reviews_exp.shape[0])

Porcentaje de valores vacios de la columna last_edited:  89.64674142146531


In [ ]:
# Se elimina la columna 'last_edited'
df_reviews_exp = df_reviews_exp.drop('last_edited', axis=1)
# df_reviews_exp.columns

In [ ]:
# Se eliminan las columnas que nos son utiles para nuestro estudio  
df_reviews_exp.drop(['user_url', 'funny', 'helpful'], axis=1, inplace=True)
df_reviews_exp.columns

In [19]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_reviews_exp.replace(['', 'null', 'None'], np.nan, inplace=True)

In [20]:
# Se verifican y se eliminan las filas duplicadas
filas_dup = df_reviews_exp.duplicated().sum()
print(filas_dup)
df_reviews_exp.drop_duplicates(inplace=True)
print(df_reviews_exp.shape)

874
(58431, 5)


In [21]:
# Se obtiene el año del review a partir de la columna 'posted'

# Se convierte 'posted' a  tipo datetime
df_reviews_exp['posted'] = pd.to_datetime(df_reviews_exp['posted'], format='Posted %B %d, %Y.', errors='coerce')

print(df_reviews_exp['posted'].value_counts()) 

posted
2014-06-21    220
2014-06-20    192
2014-06-27    171
2014-06-23    171
2013-12-26    169
             ... 
2011-03-25      1
2011-11-16      1
2011-05-28      1
2012-02-01      1
2012-11-03      1
Name: count, Length: 1644, dtype: int64


In [22]:
# Crea la columna 'year' a partir de 'posted'
df_reviews_exp['year'] = df_reviews_exp['posted'].dt.year.astype('Int64')

print(df_reviews_exp['year'].value_counts()) 


year
2014    21834
2015    18154
2013     6713
2012     1201
2011      530
2010       66
Name: count, dtype: Int64


In [23]:
print(df_reviews_exp['year'].value_counts()) 

var_nulos = (df_reviews_exp['year'].isnull().sum()) 
var_total = (df_reviews_exp.shape[0])
percent_nulos = var_nulos/var_total

print(var_nulos, var_total, percent_nulos*100)



year
2014    21834
2015    18154
2013     6713
2012     1201
2011      530
2010       66
Name: count, dtype: Int64
9933 58431 16.999537916516918


In [24]:
# Se ordena el DataFrame por 'item_id' y 'year' 

df_reviews_exp = df_reviews_exp.sort_values(['item_id', 'year'])

In [25]:
# Se elimina la columnas 'posted' que ya no es de utilidad
df_reviews_exp.drop(['posted'], axis=1, inplace=True)
df_reviews_exp.head()

,user_id,item_id,recommend,review,year
5331,76561198040188061,10,True,this game is the 1# online action game is awes...,2011
22702,maddoxx789,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,mixadance,10,True,:D,2012
43134,waspish,10,True,Good Game :D,2012
24137,76561198001699914,10,True,jueguenlooooooo,2013


In [26]:
from langdetect import detect

In [27]:
# Determinar el lenguaje en el que fueron escritos los 'review'

#df_reviews_exp['idioma'] = df_reviews_exp['review'].apply(lambda x: detect(str(x)) if pd.notna(x) else 'No detectado')


def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'idioma'
df_reviews_exp['idioma'] = df_reviews_exp['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_reviews_exp['idioma'].value_counts()
porcentaje_por_idioma = df_reviews_exp['idioma'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()


,Conteo,Porcentaje
idioma,,
en,45076,77.91%
pt,2156,3.73%
es,1257,2.17%
de,1136,1.96%
so,1000,1.73%


In [28]:
# Para eliminar los review que no estan en Ingles 'en' y se almacenara en el dataframe df_reviews_en 
df_reviews_en = df_reviews_exp[df_reviews_exp['idioma']== 'en'].copy()

print(df_reviews_exp['idioma'].value_counts()[0:5]) 
print(df_reviews_en['idioma'].value_counts()) 


idioma
en    45076
pt     2156
es     1257
de     1136
so     1000
Name: count, dtype: int64
idioma
en    45076
Name: count, dtype: int64


In [29]:
# Eliminar filas con valores nulos en 'review'
var_nulos = (df_reviews_en['review'].isnull().sum()) 
print(var_nulos)
df_reviews_en = df_reviews_en.dropna(subset=['review'])


0


In [30]:
print(df_reviews_en.info())

<class 'pandas.core.frame.DataFrame'>
Index: 45076 entries, 5331 to 32632
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    45076 non-null  object
 1   item_id    45076 non-null  object
 2   recommend  45076 non-null  bool  
 3   review     45076 non-null  object
 4   year       37223 non-null  Int64 
 5   idioma     45076 non-null  object
dtypes: Int64(1), bool(1), object(4)
memory usage: 2.1+ MB
None


In [31]:
var_nulos = (df_reviews_en['year'].isnull().sum()) 
print(var_nulos)

7853


In [32]:
# Se Eliminan los valores que no tienen un año especifico 
df_reviews_en = df_reviews_en.dropna(subset=['year'])


In [33]:
print(df_reviews_en.info())

<class 'pandas.core.frame.DataFrame'>
Index: 37223 entries, 5331 to 32632
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    37223 non-null  object
 1   item_id    37223 non-null  object
 2   recommend  37223 non-null  bool  
 3   review     37223 non-null  object
 4   year       37223 non-null  Int64 
 5   idioma     37223 non-null  object
dtypes: Int64(1), bool(1), object(4)
memory usage: 1.8+ MB
None


In [34]:
# Convertir los datos de la columna 'item_id' a formato entero
df_reviews_en['item_id'] = df_reviews_en['item_id'].astype(int) 
print(df_reviews_en.info())

<class 'pandas.core.frame.DataFrame'>
Index: 37223 entries, 5331 to 32632
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    37223 non-null  object
 1   item_id    37223 non-null  int32 
 2   recommend  37223 non-null  bool  
 3   review     37223 non-null  object
 4   year       37223 non-null  Int64 
 5   idioma     37223 non-null  object
dtypes: Int64(1), bool(1), int32(1), object(3)
memory usage: 1.6+ MB
None


### 3. ETL-ITEMS

In [35]:
# Se expandira el dataframe df_items para que quede una fila por cada recomendación 
# Y una columna por cada dato de la seria almacenada en la columna de 'items'

expanded_rows = []
for index, row in df_items.iterrows():
    user_id  = row['user_id']
    user_url = row['user_url']

    items_list = row['items']

    for it in items_list:
        # se genera diccionario del nuevo df expandido
        new_item = {    'user_id'  : user_id ,
                        'user_url' : user_url,
                        **it
                    }
        expanded_rows.append(new_item)

df_items_exp = pd.DataFrame(expanded_rows)
#df_items_exp.to_csv(r'./Datasets/items_exp.csv',index=False)

with pd.option_context('display.max_colwidth', None):
    print(df_items_exp.head(5))  

             user_id                                              user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
1  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
2  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
3  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   
4  76561197970982479  http://steamcommunity.com/profiles/76561197970982479   

  item_id                  item_name  playtime_forever  playtime_2weeks  
0      10             Counter-Strike                 6                0  
1      20      Team Fortress Classic                 0                0  
2      30              Day of Defeat                 7                0  
3      40         Deathmatch Classic                 0                0  
4      50  Half-Life: Opposing Force                 0                0  


In [36]:
df_items_exp.columns

Index(['user_id', 'user_url', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [37]:
# Se eliminan las columnas que nos son utiles para nuestro estudio
df_items_exp.drop(['user_url', 'playtime_2weeks'], axis=1, inplace=True)
df_items_exp.columns


Index(['user_id', 'item_id', 'item_name', 'playtime_forever'], dtype='object')

In [38]:
# Convertir los datos de 'item_name' a minusculas
df_items_exp['item_name'] = df_items_exp['item_name'].str.lower()
df_items_exp.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,counter-strike,6
1,76561197970982479,20,team fortress classic,0
2,76561197970982479,30,day of defeat,7
3,76561197970982479,40,deathmatch classic,0
4,76561197970982479,50,half-life: opposing force,0


In [39]:
# Se eliminan las filas duplicadas
filas_dup = df_items_exp.duplicated().sum()
print(filas_dup)
print(df_items_exp.shape)

df_items_exp.drop_duplicates(inplace=True)
print(df_items_exp.shape)

59117
(5153209, 4)
(5094092, 4)


In [ ]:
# Asumiendo que ya tienes un DataFrame llamado df y una columna llamada 'columna_de_interes'
# z_scores = zscore(df_items['playtime_forever'])
# df_items = df_items[(z_scores < 3) & (z_scores > -3)]  # Filtrar valores dentro de 3 desviaciones estándar

# Convertir los datos de la columna 'item_id' a entero
df_items['item_id'] = df_items['item_id'].astype(int)
#df_items = df_items[df_items['playtime_forever'] != 0]

### 4. ETL-GAMES

In [41]:
print(df_games.shape)
print(df_games.info())    # Se observan una gran cantidad de datos nulos



(32135, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB
None


In [42]:
print(df_games.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [43]:
# Se eliminan las filas del dataframe que tienen todos sus valores nulos
df_games = df_games.dropna(how='all')
print(df_games.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [44]:
# print(df_games['price'].value_counts())
#prices = df_games[df_games['price'] == 'Free to play']
#print(prices.info())

#price_free = df_games['price'].str.contains('Free to play', case=False, na=False).sum()

price_free = df_games['price'].str.contains('Free to play', case=False, na=False)
print(price_free)

88310     False
88311      True
88312      True
88313     False
88314     False
          ...  
120440    False
120441    False
120442    False
120443    False
120444    False
Name: price, Length: 32135, dtype: bool


In [45]:
# Se eliminan las filas con datos nulos en las columans 'id', 'release_date', 'developer' y 'price'
df_games = df_games.dropna(subset=['id', 'release_date', 'developer', 'price'])
print(df_games.info())


<class 'pandas.core.frame.DataFrame'>
Index: 27596 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22916 non-null  object 
 1   genres        27462 non-null  object 
 2   app_name      27596 non-null  object 
 3   title         27596 non-null  object 
 4   url           27596 non-null  object 
 5   release_date  27596 non-null  object 
 6   tags          27488 non-null  object 
 7   reviews_url   27596 non-null  object 
 8   specs         27316 non-null  object 
 9   price         27596 non-null  object 
 10  early_access  27596 non-null  float64
 11  id            27596 non-null  float64
 12  developer     27596 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.9+ MB
None


In [46]:

price_free = df_games['price'].str.contains('Free to play', case=False, na=False).sum()
print(price_free)

842


In [47]:
# Convertir columna 'price' en numero/float y cambiar la cadena 'Free to play' por cero
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce').fillna(0)
df_games['price'] = df_games['price'].astype(float)

print(df_games.info())

<class 'pandas.core.frame.DataFrame'>
Index: 27596 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22916 non-null  object 
 1   genres        27462 non-null  object 
 2   app_name      27596 non-null  object 
 3   title         27596 non-null  object 
 4   url           27596 non-null  object 
 5   release_date  27596 non-null  object 
 6   tags          27488 non-null  object 
 7   reviews_url   27596 non-null  object 
 8   specs         27316 non-null  object 
 9   price         27596 non-null  float64
 10  early_access  27596 non-null  float64
 11  id            27596 non-null  float64
 12  developer     27596 non-null  object 
dtypes: float64(3), object(10)
memory usage: 2.9+ MB
None


In [48]:
valores_0 = df_games['price'].value_counts().get(0, 0)
#valores_0 = df_games['price'].value_counts()
print(valores_0)

1544


In [49]:
df_games_free = df_games[df_games['price'] == 0]
print(df_games_free.head(20))

                                publisher  \
88311                    Making Fun, Inc.   
88312                        Poolians.com   
88370        Unknown Worlds Entertainment   
88501                Tripwire Interactive   
88558                     Darklight Games   
88575                         Digital Eel   
88666                Tripwire Interactive   
88679                   Octoshark Studios   
88752                               Valve   
88927                        Trion Worlds   
88963              En Masse Entertainment   
88964         Perfect World Entertainment   
88965                                None   
88969  Peter Brinson and Kurosh ValaNejad   
89030                         Three Rings   
89064     Days of Wonder, Asmodee Digital   
89077               Daybreak Game Company   
89105                      Reloaded Games   
89152         Perfect World Entertainment   
89176                          Deca Games   

                                                  genr

In [50]:
# Cambiar el nombre de la columna "id" por "item_id" y convertir a enteros
df_games = df_games.rename(columns={'id': 'item_id'})
df_games['item_id'] = df_games['item_id'].astype(int) 

print(df_games.info())

<class 'pandas.core.frame.DataFrame'>
Index: 27596 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22916 non-null  object 
 1   genres        27462 non-null  object 
 2   app_name      27596 non-null  object 
 3   title         27596 non-null  object 
 4   url           27596 non-null  object 
 5   release_date  27596 non-null  object 
 6   tags          27488 non-null  object 
 7   reviews_url   27596 non-null  object 
 8   specs         27316 non-null  object 
 9   price         27596 non-null  float64
 10  early_access  27596 non-null  float64
 11  item_id       27596 non-null  int32  
 12  developer     27596 non-null  object 
dtypes: float64(2), int32(1), object(10)
memory usage: 2.8+ MB
None


In [51]:
df_games['developer'] = df_games['developer'].astype(str).str.lower()

In [52]:
# Se obtiene el año del release_date requerido para el análisis
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
df_games['year'] = df_games['release_date'].dt.year

### 5. ANALISIS DE SENTIMIENTO

In [53]:
# Analisis de sentimientos
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargar el lexicon necesario para el análisis de sentimientos
nltk.download('vader_lexicon')

# Crear una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Convertimos la columna 'review' en tipo string para aplicar nuestro modelo de analisis de sentimiento
df_reviews_en['review'] = df_reviews_en['review'].astype(str)

# Calcula el puntaje de sentimiento para cada reseña y agrega al df una nueva columna "sentimiento"
df_reviews_en['sentimiento'] = df_reviews_en['review'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Definimos umbrales para clasificar las reseñas en positivas, neutrales y negativas
umbral_positivo = 0.3
umbral_negativo = -0.3

# Crea una nueva columna "sentimiento_etiqueta" para clasificar en positivo, neutral y negativo
df_reviews_en['sentimiento_etiqueta'] = df_reviews_en['sentimiento'].apply(lambda x: 2 if x > umbral_positivo else (0 if x < umbral_negativo else 1))



[nltk_data] Downloading package vader_lexicon to C:\Users\Diego
[nltk_data]     Osorio\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [54]:
df_reviews_en.head(4)

,user_id,item_id,recommend,review,year,idioma,sentimiento,sentimiento_etiqueta
5331,76561198040188061,10,True,this game is the 1# online action game is awes...,2011,en,0.7906,2
45506,epic_doom,10,True,The OG to CS:GO.,2013,en,0.0000,1
7801,mayshowganmore,10,True,THE BEST FPS GAME!!!!!,2014,en,0.7482,2
7967,BestinTheWorldThund3r,10,True,One of the best childhood games i have ever pl...,2014,en,0.8011,2


In [55]:
print(df_reviews_en['sentimiento_etiqueta'].value_counts())

sentimiento_etiqueta
2    24577
1     7664
0     4982
Name: count, dtype: int64


## FUNCIONES PARA LA API

### 5. developer_reviews_analysis( desarrolladora : str ): 

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [56]:
# creamos 2 dataframes solo con las columnas requeridas y hacemos un inner join
df_games_API5 = df_games[['item_id', 'developer']]
df_reviews_API5 = df_reviews_en[['item_id', 'sentimiento_etiqueta']]

df_API5 = pd.merge(df_games_API5, df_reviews_API5, on = "item_id", how = "inner")

In [57]:
# Agrupamos por developer y sentimiento, contamos los coincidentes, y ordenamos por  columnas el resultado
df_API5 = df_API5.groupby(['developer', 'sentimiento_etiqueta']).size().unstack(fill_value=0)
print(df_API5)

sentimiento_etiqueta                    0   1   2
developer                                        
07th expansion                          1   1   0
10th art studio,adventure productions   0   0   1
11 bit studios                         17  10  13
14° east                                1   0   1
17-bit                                  0   0   1
...                                    ..  ..  ..
zombie studios                          1   1   1
zootfly                                 0   0   2
zoë mode                                0   1   1
zykov eddy,xitilon                      0   1   1
高考恋爱委员会,橘子班                             0   0   1

[1569 rows x 3 columns]


In [58]:
# Agregamos indices al nuevo dataframe y mantenemos la columna developer
df_API5 = df_API5 .reset_index(drop = False)
print(df_API5)

sentimiento_etiqueta                              developer   0   1   2
0                                            07th expansion   1   1   0
1                     10th art studio,adventure productions   0   0   1
2                                            11 bit studios  17  10  13
3                                                  14° east   1   0   1
4                                                    17-bit   0   0   1
...                                                     ...  ..  ..  ..
1564                                         zombie studios   1   1   1
1565                                                zootfly   0   0   2
1566                                               zoë mode   0   1   1
1567                                     zykov eddy,xitilon   0   1   1
1568                                            高考恋爱委员会,橘子班   0   0   1

[1569 rows x 4 columns]


In [59]:
# Damos nombres apropiados a las columnas
nombre_columns = ["Developer",'Negativo', 'Neutro',"Positivo"]
df_API5.columns = nombre_columns

# Damos formato de lowercase a columna de developer
df_API5['Developer'] = df_API5['Developer'].astype(str).str.lower()

print(df_API5.head())

                               Developer  Negativo  Neutro  Positivo
0                         07th expansion         1       1         0
1  10th art studio,adventure productions         0       0         1
2                         11 bit studios        17      10        13
3                               14° east         1       0         1
4                                 17-bit         0       0         1


In [60]:
#df_API5.to_parquet("df_API5.parquet", index=False )

In [62]:
# Funcion que dando como entrada el nombre de un Developer arroja el total de reviews Negativos y Posivitos dados
def developer_reviews(desarrollador):
    
    desarrollador=desarrollador.lower()    # aseguramos lowercase para la entrada dada por el usuario

    # Filtramos para obtener solo la fila del Developer ingresado por el usuario
    fila_desarrollador = df_API5[df_API5['Developer'] == desarrollador]

    review_negativo = fila_desarrollador['Negativo'].values[0]
    review_positivo = fila_desarrollador['Positivo'].values[0]

    # Crea el diccionario para mostrar el resultado solicitado
    result = {
        desarrollador: {
            'Negativo': review_negativo,
            'Positivo': review_positivo
        }
    }

    # Para presentar el resultado en el formato solicitado
    result = {clave: [f"{subclave} = {valor}" for subclave, valor in subdiccionario.items()] for clave, subdiccionario in result.items()}

    return result

# Llamada a la funcion
developer_reviews('tobyfox')

{'tobyfox': ['Negativo = 18', 'Positivo = 67']}

In [63]:
df_API5.to_parquet("df_game_review_API5.parquet", index=False )

### 4. best_developer_year( año : int ): 
Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [64]:
df_reviews_en.head(5)

,user_id,item_id,recommend,review,year,idioma,sentimiento,sentimiento_etiqueta
5331,76561198040188061,10,True,this game is the 1# online action game is awes...,2011,en,0.7906,2
45506,epic_doom,10,True,The OG to CS:GO.,2013,en,0.0000,1
7801,mayshowganmore,10,True,THE BEST FPS GAME!!!!!,2014,en,0.7482,2
7967,BestinTheWorldThund3r,10,True,One of the best childhood games i have ever pl...,2014,en,0.8011,2
8519,76561198072207162,10,True,People still play this! Siq game,2014,en,0.4003,2


In [65]:
# creamos 2 dataframes con las columnas requeridas y hacemos un inner join
df_reviews_API4 = df_reviews_en[['item_id', 'recommend', 'year']]
df_games_API4 = df_games[['item_id', 'developer']]

df_API4 = pd.merge(df_games_API4, df_reviews_API4, on = "item_id", how = "inner")

# filtramos para obtener solo las recomendaciones positivas
df_API4 = df_API4[df_API4['recommend'] == True]

# Nos quedamos solo con las 2 columnas requeridas
df_API4 = df_API4[['developer', 'year']]

print(df_API4.head())

#df_API4.to_parquet(r'df_API4.parquet',index=False)


  developer  year
0     valve  2011
1     valve  2011
2     valve  2012
3     valve  2012
4     valve  2013


In [66]:
def best_developer(anio):

    # Filtramos para obtener solo los datos del año indicado
    fila_year = df_API4[df_API4['year'] == anio]

    # Contamos la cantidad de recomendaciones True
    recomen = fila_year['developer'].value_counts()
    recomen = recomen.head(3)

    return recomen

best_developer(2015)


developer
valve                    1754
facepunch studios         382
smartly dressed games     193
Name: count, dtype: int64

In [67]:
df_API4.to_parquet("df_game_review_API4.parquet", index=False )

### 3. userForGenre( genero : str ): 

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
#print(df_games.info())
#df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
#df_games['year'] = df_games['release_date'].dt.year
#df_games = df_games.dropna(subset=['year'])
#print(df_games.info())


In [68]:
df_games['item_id'] = df_games['item_id'].astype(int) 
df_items_exp['item_id'] = df_items_exp['item_id'].astype(int) 

In [69]:
df_games_API3 = df_games[['item_id','genres','year']]
df_items_API3 = df_items_exp[['user_id', 'item_id', 'playtime_forever']]
df_API3 = pd.merge(df_games_API3, df_items_API3, on='item_id', how = "inner")

#print(df_API3.head())

In [70]:
# Funcion para resolver problema API NO. 3: userForGenre
def userForGenre(genero:str, df_API3):
    
    # Filtramos el dataframe por genero
    df_filtrado = df_API3[df_API3['genres'].apply(lambda lista: genero in map(str.lower, lista) if isinstance(lista, list) else False)]

    # Agrupamos por 'user_id' sumamos las horas jugadas para obtener el jugador con max de horas jugadas
    df_horas_jugadas_1 = df_filtrado.groupby(['user_id'])['playtime_forever'].sum().reset_index()

    # Buscamos el usuario con la máxima suma de horas jugadas
    index_playtime_max = df_horas_jugadas_1['playtime_forever'].idxmax()
    usuario_playtime_max = df_horas_jugadas_1.loc[index_playtime_max, 'user_id']

    # Agrupamos por 'user_id' y por 'year' y sumamos las horas jugadas para poder discretizar por años
    df_horas_jugadas_2 = df_filtrado.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

    # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
    df_usuario = df_horas_jugadas_2[df_horas_jugadas_2['user_id'] == usuario_playtime_max]

    # Creamos el formato para mostrar el resutado 
    resultado_final = [{'Año': int(row['year']), 'Horas': int(row['playtime_forever'])} for _, row in df_usuario.iterrows()]

    return {"Usuario con más horas jugadas para Género {}:".format(genero): usuario_playtime_max, "Horas jugadas": resultado_final}

# Ejemplo de uso
userForGenre('racing',df_API3)
    

{'Usuario con más horas jugadas para Género racing:': 'DownSyndromeKid',
 'Horas jugadas': [{'Año': 1994, 'Horas': 0},
  {'Año': 1995, 'Horas': 318},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1999, 'Horas': 0},
  {'Año': 2005, 'Horas': 91},
  {'Año': 2006, 'Horas': 74553},
  {'Año': 2007, 'Horas': 147814},
  {'Año': 2008, 'Horas': 147707},
  {'Año': 2010, 'Horas': 0},
  {'Año': 2011, 'Horas': 295361},
  {'Año': 2013, 'Horas': 74271},
  {'Año': 2014, 'Horas': 990},
  {'Año': 2015, 'Horas': 1910},
  {'Año': 2016, 'Horas': 185}]}

In [71]:
df_API3.to_parquet("df_game_items_API3.parquet", index=False )

### 2. userdata( User_id : str ): 

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [ ]:
#print(df_games.columns)
#print(df_reviews_en.columns)
#print(df_items_exp.columns)

In [74]:
df_games_API2 = df_games[['item_id','price']]
df_items_API2 = df_items_exp[['user_id', 'item_id']]
df_reviews_API2 = df_reviews_en[['user_id', 'recommend']]

df_API2 = pd.merge(df_games_API2, df_items_API2, on='item_id', how = "inner")

# Filtro para tomar solo las filas con recommend = True
df_review_true_API2 = df_reviews_API2[df_reviews_API2['recommend'] == True] 
#print(review_true.info())

def userdata(usuario:str):
    
    # filtro con valor de usuario en tabla que combina games e items
    filtro_usuario = df_API2['user_id'].str.contains(usuario, case=False, na=False)
    df_filtrado = df_API2[filtro_usuario]
    # print(df_filtrado.head())

    # Total gastado por el usuario
    gastado = df_filtrado['price'].sum()
    #print("gastado:", gastado)

    # Total de items
    total_items= df_filtrado.shape[0]
    #print("total items:", total_items)

    # Calculo de % de recomendacion
    # Filtro con valor de usuario en tabla review
    review_usuario = df_review_true_API2[df_review_true_API2['user_id'] == usuario] 

    items_total = (df_filtrado['item_id'].value_counts())   # total de items
    #print("items:", items_total.shape[0])

    #review_usuario = review_true[review_true['user_id'] == usuario]   # total de recomendados
    #print(review_usuario.head())
    
    porcentaje_recomen = (review_usuario.shape[0]*100/items_total.shape[0])

    # Creamos el formato para mostrar el resutado
    resultado = (f'{{"Usuario": "{usuario}", "Dinero gastado": {gastado:.2f} USD, "% de recomendación": {porcentaje_recomen:.2f}%, "Cantidad de items": {total_items}}}')

    return resultado

# Ejemplo de uso
userdata('DownSyndromeKid')

# df_games_anio.to_parquet(r'./Datasets/df_games_developer.parquet',index=False)

'{"Usuario": "DownSyndromeKid", "Dinero gastado": 5768.70 USD, "% de recomendación": 0.00%, "Cantidad de items": 1218}'

In [75]:
df_API2.to_parquet(r'./Datasets/df_games_items_API2.parquet',index=False)
df_review_true_API2.to_parquet(r'./Datasets/df_review_true_API2.parquet',index=False)

### 1. def developer ( desarrollador : str ): 

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [76]:
df_games_API1 = df_games[['developer', 'price', 'year']]

#empresa = 'monster games'

def developer(empresa):
    # filtrado
    df_games_filtra = df_games_API1[df_games_API1['developer'] == empresa]
    #print(df_games_filtra.head(100))

    #df_filtrado_kotoshiro = df_games_API1[df_games_API1['developer'] == 'kotoshiro']

    # Agrupado por año
    df_games_anio = df_games_filtra.groupby(['year'])['price'].size().reset_index() 
    #print(df_games_anio.head(10))

    # Filtro para obtener solo los contenidos free
    df_games_free = df_games_filtra[df_games_filtra['price'] == 0]
    #print(df_games_free.head(20))

    # Agrupamos por año
    df_games_free_anio = df_games_free.groupby(['year'])['price'].size().reset_index()
    #print(df_games_free_anio.head(100))

    # Se obtiene el porcetanje de conteido free
    df_games_anio['percent_free']= df_games_free_anio['price']*100 / df_games_anio['price']
    #print(df_games_anio)

    # Damos el formato solicitado
    df_games_anio.columns = ['Año', 'Cantidad de Items', 'Contenido Free']
    df_games_anio['Año'] = df_games_anio['Año'].map('{:.0f}'.format)
    df_games_anio['Contenido Free'] = df_games_anio['Contenido Free'].map('{:.0f}%'.format)

    # Imprimir la tabla formateada con nombres de columnas personalizados
    print(tabulate(df_games_anio, headers='keys', tablefmt='pretty', showindex=False))

    return df_games_anio

developer('monster games')

df_games_API1.to_parquet(r'./Datasets/df_games_API1.parquet',index=False)

+------+-------------------+----------------+
| Año  | Cantidad de Items | Contenido Free |
+------+-------------------+----------------+
| 2016 |        33         |      42%       |
| 2017 |        17         |      35%       |
+------+-------------------+----------------+


## Modelo de aprendizaje automático

In [77]:
print(df_games.columns)
print(df_items_exp.columns)
print(df_reviews_en.columns)

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer', 'year'],
      dtype='object')
Index(['user_id', 'item_id', 'item_name', 'playtime_forever'], dtype='object')
Index(['user_id', 'item_id', 'recommend', 'review', 'year', 'idioma',
       'sentimiento', 'sentimiento_etiqueta'],
      dtype='object')


In [78]:
# Seleccionamos las columnas necesarias de los dataframes de interes para el sistema de recomendacion 
#df_reviews = pd.read_parquet("reviews_eda.parquet")
#df_games = pd.read_parquet("games_eda.parquet")

df_games_ML = df_games[["item_id","title"]]
df_reviews_ML = df_reviews_en[["user_id","item_id","recommend"]]

# Hacemos el merge entre los dos dataframes y ns quedamos solo con las columna necesarias 
df_ML = pd.merge(df_reviews_ML,df_games_ML, on="item_id", how="inner")
df_ML = df_ML[["user_id","title","recommend"]]

#df_ML_filtered = df_ML[df_ML['recommend'] == True]
df_ML = df_ML.astype(str)
#df.to_parquet("df_ML.parquet", index= False)

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

#user_id = "76561197970982479"

# Funcion para implementar el calculo del cosine_similarity y recomendar los 5 juegos solicitados
def recomendacion_usuario(user_id :str):

    # Crear una matriz de recomendaciones donde las filas son usuarios y las columnas son juegos
    user_game_matrix = pd.crosstab(df_ML['user_id'], df_ML['title'])

    try:
        # Encuentra el índice del usuario en la matriz
        user_index = user_game_matrix.index.get_loc(user_id)
    except KeyError:
        print(f"El usuario {user_id} no está presente en los datos.")
        return None

    # Calcula la similitud de coseno entre los usuarios
    cosine_similarities = cosine_similarity(user_game_matrix, user_game_matrix)

    # Obtén las similitudes de coseno para el usuario dado
    similar_users = cosine_similarities[user_index]

    # Encuentra los juegos que el usuario no ha calificado
    games_played = user_game_matrix.loc[user_id]
    unrated_games = games_played[games_played == 0].index

    # Calcula las puntuaciones de recomendación sumando las similitudes de usuarios para los juegos no calificados
    recommendation_scores = user_game_matrix.loc[:, unrated_games].multiply(similar_users, axis=0).sum(axis=0)

    # Ordena las recomendaciones por puntuación descendente
    recommendations = recommendation_scores.sort_values(ascending=False).index.tolist()

    # Limita las recomendaciones a los primeros 5 juegos
    top_recommendations = recommendations[:5]
    #print(top_recommendations)

    return top_recommendations

# Ejemplo de uso:
# usuario_a_recomendar es el ID del usuario para el que quieres obtener recomendaciones
recomendacion_usuario('76561197970982479')


['Team Fortress 2', "Garry's Mod", 'Counter-Strike: Global Offensive', 'Warframe', 'Left 4 Dead 2']


['Team Fortress 2',
 "Garry's Mod",
 'Counter-Strike: Global Offensive',
 'Warframe',
 'Left 4 Dead 2']